# FHIR ❤️ Dashboard

#### Fhirbase Setup

In [23]:
import psycopg2
conn = psycopg2.connect("dbname=fhirbase")
cur = conn.cursor()

## Requêtes

### Durée moyenne de séjour

In [15]:
query = """
            SELECT AVG((e.resource#>>'{period, end}')::date - (e.resource#>>'{period, start}')::date) AS len 
            FROM encounter e 
            WHERE e.resource#>>'{class, code}' = 'inpatient' 
            OR e.resource#>>'{class, code}' = 'emergency'
        """
cur.execute(query)
cur.fetchone()

(Decimal('12.7630480167014614'),)

### Nombre d'entrées et sorties classées par mois

**Nombre d'entrées**

In [16]:
query_inpatients = """
            SELECT 
                date_trunc('month', (e.resource#>>'{period, start}')::date) as date, 
                count(*) as entrees 
            FROM encounter e 
            WHERE 
                e.resource#>>'{class, code}' = 'inpatient' 
                OR e.resource#>>'{class, code}' = 'emergency' 
            GROUP BY date 
            ORDER BY date DESC 
            LIMIT 10;
            """

cur.execute(query_inpatients)
[(str(n[0]), n[1]) for n in cur]

[('2018-08-01 00:00:00+02:00', 4),
 ('2018-07-01 00:00:00+02:00', 9),
 ('2018-06-01 00:00:00+02:00', 4),
 ('2018-05-01 00:00:00+02:00', 8),
 ('2018-04-01 00:00:00+02:00', 6),
 ('2018-03-01 00:00:00+01:00', 5),
 ('2018-02-01 00:00:00+01:00', 3),
 ('2018-01-01 00:00:00+01:00', 8),
 ('2017-12-01 00:00:00+01:00', 6),
 ('2017-11-01 00:00:00+01:00', 2)]

**Nombre de sorties**

In [17]:
query_outpatients = """
            SELECT 
                date_trunc('month', (e.resource#>>'{period, end}')::date) as date, 
                count(*) as sorties 
            FROM encounter e 
            WHERE 
                e.resource#>>'{class, code}' = 'inpatient' 
                OR e.resource#>>'{class, code}' = 'emergency' 
            GROUP BY date 
            ORDER BY date DESC 
            LIMIT 10;
            """
cur.execute(query_outpatients)
[(str(n[0]), n[1]) for n in cur]

[('2018-08-01 00:00:00+02:00', 4),
 ('2018-07-01 00:00:00+02:00', 9),
 ('2018-06-01 00:00:00+02:00', 5),
 ('2018-05-01 00:00:00+02:00', 7),
 ('2018-04-01 00:00:00+02:00', 6),
 ('2018-03-01 00:00:00+01:00', 5),
 ('2018-02-01 00:00:00+01:00', 3),
 ('2018-01-01 00:00:00+01:00', 8),
 ('2017-12-01 00:00:00+01:00', 6),
 ('2017-11-01 00:00:00+01:00', 2)]

### Nombre de lits disponible par Service
*Pas de résultats dans la base de données par défaut Fhirbase + MIMIC*

In [18]:
query = """
            SELECT 
                l.resource#>>'{operationalStatus, display}' AS bedStatus, 
                ll.resource#>>'{name}' AS service, 
                count(*)
            FROM 
                location l 
            JOIN 
                location ll ON l.resource#>>'{partOf, identifier}' = ll.resource#>>'{identifier}' 
            WHERE 
                l.resource#>>'{status, code}' = 'active' 
                AND l.resource#>>'{physicalType, code}' = 'bd'  
                AND ll.resource#>>'{physicalType, code}' = 'wd' 
            GROUP BY bedStatus, service;
"""
cur.execute(query)
[n for n in cur]

[]

### Temps d'attente moyen aux urgences (en direct)
*Pas de résultats dans la base de données par défaut Fhirbase + MIMIC*

**Deux méthodes**

In [27]:
# using the EpisodeOfCare resource
query = """
            SELECT 
                AVG(now() - (e.resource#>>'{period, start}')::date) AS len 
            FROM episodeofcare e 
            WHERE 
                e.resource#>>'{status, code}' = 'onhold' 
                OR e.resource#>>'{status, code}' = 'waitlist'
    """
cur.execute(query)
[n for n in cur]

[(None,)]

In [20]:
# using the Encounter resource
query = """
            SELECT 
                AVG(now() - (e.resource#>>'{period, start}')::date) AS len 
            FROM Encounter e 
            WHERE 
                e.resource#>>'{class, code}' = 'PRENC' 
    """
cur.execute(query)
[n for n in cur]

[(None,)]

### Nombre de consultations
*La mock database ne permet pas de l'ordonner par specialites (pas de CareTeam / Location renseigne)*

In [26]:
query = """
            SELECT 
                date_trunc('month', (e.resource#>>'{period, end}')::date) AS date, count(*) 
            FROM encounter e 
            WHERE 
                e.resource#>>'{class, code}' = 'ambulatory' 
            GROUP BY date 
            ORDER BY date DESC
            LIMIT 10;
"""
cur.execute(query)
[(str(n[0]), n[1]) for n in cur]

[('2018-08-01 00:00:00+02:00', 13),
 ('2018-07-01 00:00:00+02:00', 37),
 ('2018-06-01 00:00:00+02:00', 37),
 ('2018-05-01 00:00:00+02:00', 42),
 ('2018-04-01 00:00:00+02:00', 43),
 ('2018-03-01 00:00:00+01:00', 33),
 ('2018-02-01 00:00:00+01:00', 27),
 ('2018-01-01 00:00:00+01:00', 37),
 ('2017-12-01 00:00:00+01:00', 30),
 ('2017-11-01 00:00:00+01:00', 32)]